In [1]:
#Import packages
import SAP_from_SPH as SAP
import numpy as np
from scipy.optimize import fsolve
from importlib import reload
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
reload(SAP)

<module 'SAP_from_SPH' from 'C:\\Users\\45177\\PythonEverything\\Princeton\\2024\\Hamid_Hypar\\SAP_from_SPH.py'>

In [2]:
def sap_response(b,c,theta,n_b,n_c,H_w,T_w,warp_r,path,press_file):
    

    #Parameters
    gvty_water = 9.7706 # KN per m^3, density times GA g 

    # Parameters that define Hypar Geometry
    #Unit KN & m
    b1 = b #longitudinal dimension of lower panel
    b2 = b
    c1 = c #Left transverse dimension
    c2 = c
    A = (b1+b2)*(c1+c2) #Projected area
    r = warp_r*A #Rise

    theta_star = theta * np.pi / 180 # angle of inclination
    h_star = (b1+b2)*np.sin(theta_star)#Height

    #FEM discretization
    n_b1 = n_b #Number of quad elements along longitudinal direction of lower panel
    n_b2 = n_b #Number of quad elements along longitudinal direction of upper panel

    n_c1 = n_c #Number of quad elements along transverse direction of left panel
    n_c2 = n_c #Number of quad elements along transverse direction of right panel

    h_off = 0.3 #Hinge offset (relative to the vertex)

    #Column settings
    col_b = 1.2 #Column side length
    col_h = 1.2 #Column side length



    #File name, API Path
    APIPath = path
    #APIPath = r'C:\Users\45177\OneDrive - Princeton University\Gaoyuan\2021Fall\Research\ParametricStudy2021\3D\Hypar\0.03_W\H1\T6\measuretool'


    #Pressure File from SPH
    press_file = press_file

    #Simulation time
    num_dt = 1200 #number of time steps
    time_step = 0.05 #0.1s time step

    #Structure's position to the slope top
    d_to_stop = 4



    #%%
    ######################
    # Wave Settings      #   
    ######################

    Hw = H_w #Wave height for goda, doesn't matter here
    dw = 4 #Inundation
    d = 9 #Water depth
    g = gvty_water #gravitational accel.
    T = T_w #Wave period

    Project_Name ='Hypar'
    #Slope value
    bot_x = 80
    top_x = 116
    slope = (d-dw)/(bot_x-top_x)

    def eq_k (k,d=d,g=g):
        return 2*np.pi/T - np.sqrt(g*k * np.tanh(k*d))
    def wavenum_k(T,d):
        return abs(fsolve(eq_k,1))
    def lbd(T,d,g):
        k = abs(fsolve(eq_k,1))
        return k,2*np.pi / k

    Lw = float((lbd(T,d,g))[1]) #Wavelength

    if (5*Hw-d_to_stop) >= (d-dw)/slope:
        hb = d
    elif (5*Hw-d_to_stop) <= 0:
        hb = dw
    else:
        hb =dw + (5*Hw-d_to_stop)*slope



    #%%

    SAP_Model,area_num,node_num,my_sapobj = SAP.Create_SAP_Hypar_Sea(Project_Name,APIPath,n_c1,n_c2,n_b1,n_b2,theta_star,r,c1,c2,b1,b2,h_off,col_b,col_h,press_file,num_dt,time_step,Hw,dw,hb,d,Lw,E_c = 4.4*10**7,gamma_shell = 23.52, E_shell = 4.4*10**7, U_shell = 0.2, T_E_shell = 9.9e-06,t_shell = 0.25)

'''   #%%
    SAP.Run_Sap(SAP_Model)
    #%%
    ret = SAP_Model.View.RefreshView(0, False)

    #%%
    reload(SAP)
    #%%
    shell_res,col_res = SAP.Post_Sap(SAP_Model,area_num,node_num)
    #%%
    res_array=np.vstack((shell_res,col_res))
    #%%
    columns = ["SPH-TH","SPH-TH(no damping)","DEAD-ST","SPH-ST","GODA-ST","INUNDATION-ST"]
    index = ["Shell-Max Moment","Shell-Max Shear","Shell-Max Comp.","Shell-Max Tension","Maximum shell disp","Col-Shear","Col-Base Moment","Col-Axial(negative-com)"]
    df = pd.DataFrame(res_array,  index,columns)
    print("response")
    #%%
    df.to_excel("Structural_H{}_T{}_w{}_b{}.xlsx".format(Hw,T,warp_r,b))  
    #%%
    ret = my_sapobj.ApplicationExit(False)

    SapModel = None

    mySapObject = None'''


'   #%%\n    SAP.Run_Sap(SAP_Model)\n    #%%\n    ret = SAP_Model.View.RefreshView(0, False)\n\n    #%%\n    reload(SAP)\n    #%%\n    shell_res,col_res = SAP.Post_Sap(SAP_Model,area_num,node_num)\n    #%%\n    res_array=np.vstack((shell_res,col_res))\n    #%%\n    columns = ["SPH-TH","SPH-TH(no damping)","DEAD-ST","SPH-ST","GODA-ST","INUNDATION-ST"]\n    index = ["Shell-Max Moment","Shell-Max Shear","Shell-Max Comp.","Shell-Max Tension","Maximum shell disp","Col-Shear","Col-Base Moment","Col-Axial(negative-com)"]\n    df = pd.DataFrame(res_array,  index,columns)\n    print("response")\n    #%%\n    df.to_excel("Structural_H{}_T{}_w{}_b{}.xlsx".format(Hw,T,warp_r,b))  \n    #%%\n    ret = my_sapobj.ApplicationExit(False)\n\n    SapModel = None\n\n    mySapObject = None'

In [3]:
path = r'C:\Users\45177\PythonEverything\Princeton\2024\Hamid_Hypar\Rn050'
press_file = r'C:\Users\45177\PythonEverything\Princeton\2024\Hamid_Hypar\Rn050\Rn0.50_Ordered.csv'

#Parameters
b = 2.5 #m
c = b
theta = 90 #upright
n_b = 15 
n_c = n_b
H_w  =3 #Dont matter
T_w = 3 #Dont matter
warp_r = 2.5/(2*b)**2

In [4]:

sap_response(b,c,theta,n_b,n_c,H_w,T_w,warp_r,path,press_file)
        

<ipython-input-4-f1ffeaf9ecdd>:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,2